In [1]:
!pip3 install pystac > x
!pip3 install robosat.pink > x

ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [0]:
from pystac import (Catalog, CatalogType, Item, Asset, LabelItem, Collection)
from urllib.parse import urlparse
import requests
from pystac import STAC_IO
import csv

def my_read_method(uri):
    parsed = urlparse(uri)
    if parsed.scheme.startswith('http'):
        return requests.get(uri).text
    else:
        return STAC_IO.default_read_text_method(uri)

STAC_IO.read_text_method = my_read_method

In [0]:
train1_cat = Catalog.from_file('https://drivendata-competition-building-segmentation.s3-us-west-1.amazonaws.com/train_tier_1/catalog.json')
train2_cat = Catalog.from_file('https://drivendata-competition-building-segmentation.s3-us-west-1.amazonaws.com/train_tier_2/catalog.json')
test_cat = Catalog.from_file('https://drivendata-competition-building-segmentation.s3-us-west-1.amazonaws.com/test/catalog.json')

In [4]:
%%shell
rm -rf data
mkdir -p data/tif data/geojson data/urls

In [0]:
def one_gig_geojson():
  tif = 'https://drivendata-competition-building-segmentation.s3-us-west-1.amazonaws.com/train_tier_1/mon/f15272/f15272.tif'
  geojson = 'https://drivendata-competition-building-segmentation.s3-us-west-1.amazonaws.com/train_tier_1/mon/f15272-labels/f15272.geojson'
  with open('data/urls/urls.csv', mode='+w') as url_file:
    url_writer = csv.writer(url_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)    
    url_writer.writerow([tif, geojson]) 

In [0]:
def store_dataset_urls(dataset_obj):
  cols = {cols.id:cols for cols in dataset_obj.get_children()}
  image_url = ''
  label_url = ''

  with open('data/urls/urls.csv', mode='+w') as url_file:
      url_writer = csv.writer(url_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)    
      for k, v in cols.items():
        for i in v.get_all_items():
            if 'label' in i.id:
              label_url = i.make_asset_hrefs_absolute().assets['labels'].href
            else: 
              image_url = i.make_asset_hrefs_absolute().assets['image'].href
            
            if image_url and label_url:
              url_writer.writerow([image_url, label_url])
              image_url, label_url = '', ''
     

In [57]:
# store_dataset_urls(train1_cat)
one_gig_geojson()

exist


In [0]:
%%shell
bash robosat_shell.sh config
while IFS=, read -r tif_url geojson_url
do
    bash robosat_shell.sh clean
    bash robosat_shell.sh preprocessing $tif_url $geojson_url 
    bash robosat_shell.sh train
done < data/urls/urls.csv
